In [1]:
!pip install gensim nltk
!wget https://raw.githubusercontent.com/bakrianoo/aravec/master/utilities.py
# Download via terminal commands
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip"
!unzip "full_grams_cbow_100_twitter.zip"
!gdown 1pHt2cR21oQyeFydiYfF3lehKPm6qon5a
!gdown 1dgZvhDEp0xHvu0jCLNNW_s1mAfvnM8jU

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-01-06 12:16:27--  https://raw.githubusercontent.com/bakrianoo/aravec/master/utilities.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2412 (2.4K) [text/plain]
Saving to: ‘utilities.py’

utilities.py        100%[===================>]   2.36K  --.-KB/s    in 0s      

2023-01-06 12:16:28 (38.1 MB/s) - ‘utilities.py’ saved [2412/2412]

--2023-01-06 12:16:28--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connec

In [2]:
import gensim
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import to_categorical 
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
from nltk import ngrams
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import sys
import argparse
from utilities import * 

In [3]:
model_t = gensim.models.Word2Vec.load("full_grams_cbow_100_twitter.mdl")
print("We've",len(model_t.wv.index2word),"vocabularies")

We've 1476715 vocabularies


In [4]:
nltk.download('stopwords')
nltk.download('punkt')
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ♥️'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
stop_words = list(set(stopwords.words('arabic')))
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['حدَث', 'سين', 'إذما', 'تانِك', 'جلل', 'حرى', 'ما أفعله', 'كاد', 'إياك', 'ليت', 'كانون', 'أيّ', 'هي', 'معاذ', 'ؤ', 'سرعان', 'كاف', 'مع', 'حزيران', 'صاد', 'أُفٍّ', 'بئس', 'مادام', 'قاطبة', 'ولا', 'من', 'سبع', 'ذِه', 'صبر', 'مايو', 'تشرين', 'ست', 'ثلاثاء', 'تخذ', 'كيفما', 'هيّا', 'تسعون', 'لكن', 'تارة', 'لما', 'بما', 'هم', 'ألفى', 'كأنّ', 'كرب', 'أرى', 'حيَّ', 'فيه', 'ظ', 'تينك', 'حبذا', 'ليستا', 'أنتما', 'أنشأ', 'ح', 'غداة', 'إلَيْكَ', 'جيم', 'حاشا', 'بك', 'إنه', 'لهم', 'عشر', 'هن', 'هلم', 'ثماني', 'عما', 'كلاهما', 'صباح', 'فمن', 'ارتدّ', 'ما انفك', 'لاسيما', 'لكيلا', 'ضحوة', 'ذوا', 'وإذ', 'مكانكما', 'خلافا', 'أمام', 'حبيب', 'هَؤلاء', 'إيه', 'حاء', 'قام', 'ع', 'أن', 'كيت', 'طَق', 'ستمئة', 'ليس', 'س', 'مكانكنّ', 'سمعا', 'ذلك', 'ز', 'أيضا', 'أفريل', 'أمس', 'سبتمبر', 'هلا', 'هَذِه', 'كأيّن', 'منه', 'تلك', 'باء', 'كسا', 'لكي', 'صهٍ', 'أوّهْ', 'قطّ', 'عامة', 'لعل', 'مئة', 'طاق', 'بكما', 'أفٍّ', 'حاي', 'دونك', 'أيّان', 'أمد', 'أخبر', 'واحد', 'تسعين', 'وُشْكَانَ', 'أنّى', 'سرا', 'لام', 'شبه',

In [5]:
train_neg = pd.read_csv('train_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None)

In [6]:
train_pos = pd.read_csv('train_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None)

In [7]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, ' ', str(text))
    return text

In [8]:
def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [9]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    text = re.sub(r'\s*[A-Za-z]+\b', ' ' , text)
    text = re.sub('[a-zA-Z0-9_]|#|http\S+', '', text)

    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

In [10]:
train = {}
for ind in train_neg.index:
  train[train_neg[1][ind]] = 0
  
for ind in train_pos.index:
  train[train_pos[1][ind]] = 1
  # print(train_neg[1][ind])

train_processed = {}
for item in train.items():
  train_processed[clean_text(item[0])]=item[1]
  

In [11]:
train_data = pd.DataFrame(train_processed.items(), columns=['sentence', 'result'])

In [12]:
train_x = train_data['sentence']
train_y = train_data['result']

In [13]:
model_t.wv[word_tokenize(train_x[0])[0]]

array([-1.0393867 , -2.692959  , -2.8078718 , -0.5087593 , -0.37763473,
        4.6773467 , -1.7567209 ,  1.1404183 ,  0.76146394,  1.4852837 ,
       -0.7773874 , -1.8126562 ,  2.3733802 , -0.35520363, -0.7767986 ,
        0.03666545, -1.7434844 , -2.8296118 ,  0.82004476,  0.98278093,
       -1.0088948 ,  0.43707576, -3.4607565 , -1.3876959 ,  1.5455184 ,
       -2.588847  , -0.4333248 , -1.0799776 , -0.772893  , -3.4521718 ,
       -0.34243122, -0.9202695 ,  0.29473847,  0.7738185 ,  1.8075006 ,
       -0.71450454,  0.7472018 ,  1.1443127 , -0.4125856 ,  0.54402256,
        0.9936972 , -0.12630965, -0.402824  ,  3.3131452 ,  0.69678944,
        2.3794215 , -0.48752823, -1.8925719 ,  1.288882  ,  2.4663148 ,
       -0.53195786, -1.95194   , -2.6357644 ,  2.8611379 ,  1.4189676 ,
        3.0073807 , -2.3403857 ,  0.24227415,  2.106204  , -1.093221  ,
        1.5740651 ,  0.41204953, -1.510692  ,  1.51844   , -1.7178947 ,
       -5.33904   , -2.7482631 , -1.7761682 ,  1.3562261 ,  1.75

In [16]:
train_x[0]

'اعترف ان بتس كانو شوي شوي يجيبو راسي اليوم بالزايد'

In [17]:
len(train_x.values)
train_x_input = np.zeros((len(train_x.values),100))

for i in range(len(train_x.values)):
  vec = np.zeros((100,))
  for word in train_x[i].split():
    if word in model_t.wv:
      vec+=model_t.wv[word]
  train_x_input[i]=vec

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu',input_shape=(100,)),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [15]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
#               optimizer=tf.keras.optimizers.Adam(1e-4),


In [18]:
tf.config.run_functions_eagerly(True)

In [19]:
history = model.fit(x=train_x_input,y=train_y,epochs=10,batch_size=8)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
3596/3596 [==============================] - 72s 20ms/step - loss: 0.6844 - accuracy: 0.6122
Epoch 2/10
3596/3596 [==============================] - 70s 19ms/step - loss: 0.6302 - accuracy: 0.6588
Epoch 3/10
3596/3596 [==============================] - 72s 20ms/step - loss: 0.6237 - accuracy: 0.6648
Epoch 4/10
3596/3596 [==============================] - 70s 19ms/step - loss: 0.6203 - accuracy: 0.6691
Epoch 5/10
3596/3596 [==============================] - 70s 19ms/step - loss: 0.6173 - accuracy: 0.6715
Epoch 6/10
3596/3596 [==============================] - 69s 19ms/step - loss: 0.6169 - accuracy: 0.6707
Epoch 7/10
3596/3596 [==============================] - 74s 21ms/step - loss: 0.6171 - accuracy: 0.6704
Epoch 8/10
3596/3596 [==============================] - 70s 20ms/step - loss: 0.6149 - accuracy: 0.6709
Epoch 9/10
3596/3596 [==============================] - 70s 19ms/step - loss: 0.6126 - accuracy: 0.6745
Epoch 10/10
3596/3596 [==============================] - 71s 20m

In [20]:
!gdown 1x2bNmaWCI-ji1-JLSI6N22tIINVv5yhD
!gdown 1L6pLW10tTO1ZM7JZU4MroPABZxAPKPh7

Downloading...
From: https://drive.google.com/uc?id=1x2bNmaWCI-ji1-JLSI6N22tIINVv5yhD
To: /content/test_Arabic_tweets_negative_20190413.tsv
100% 696k/696k [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1L6pLW10tTO1ZM7JZU4MroPABZxAPKPh7
To: /content/test_Arabic_tweets_positive_20190413.tsv
100% 776k/776k [00:00<00:00, 120MB/s]


In [21]:
test_neg = pd.read_csv('test_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None)

In [22]:
test_pos = pd.read_csv('test_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None)

In [23]:
test = {}
for ind in test_neg.index:
  test[test_neg[1][ind]] = 0
  
for ind in test_pos.index:
  test[test_pos[1][ind]] = 1
  # print(train_neg[1][ind])

In [24]:
test_processed = {}
for item in test.items():
  test_processed[clean_text(item[0])]=item[1]
  

In [25]:
test_data = pd.DataFrame(test_processed.items(), columns=['sentence', 'result'])

In [26]:
test_x = test_data['sentence']
test_y = test_data['result']

In [27]:
len(test_x.values)
test_x_input = np.zeros((len(test_x.values),100))

for i in range(len(test_x.values)):
  vec = np.zeros((100,))
  for word in test_x[i].split():
    if word in model_t.wv:
      vec+=model_t.wv[word]
  test_x_input[i]=vec

In [28]:
score = model.evaluate(test_x_input, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6121582984924316
Test accuracy: 0.6681262850761414
